In [1]:
# Credit goes to https://sundnes.github.io/solving_odes_in_python/
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.rcParams.update(mpl.rcParamsDefault)
#plt.rcParams.update({'text.usetex': True}) # import to make plots nicer
#%config InlineBackend.figure_format = 'svg' # import to make plots nicer
from scipy.integrate import solve_ivp
from matplotlib.pyplot import figure
from ipywidgets import interactive
from matplotlib.gridspec import GridSpec

#1) a) Code with the 2D Goldbetter - interactive

In [2]:
def show_sol(k1 = 9.18, k3 = 0.12 , k2 = 10, r1 = 2.04, r2 = 9.34, r3 = 0.56, r4 = 1.84, Km1 = 0.46, Km2 = 9.34, Km3 = 1.26, Km4 = 0.18,Dt = 1.26,  ACt = 10,  PDEt = 9.66, Et = 2.04):
    def cAMP_PDEp_ACp(t, x):
        cAMP = x[0]
        PDEp = x[1]

        def G(u, v, J, K): # equation (4)
                numerator = 2 * u * K # numerator of the fraction
                den_1 = v - u + v * J + u * K # first part of the denominator
                den_2 = -4 * (v - u) * u * K # second part of the denominator
                radicand = den_1**2 + den_2 # expression under the square root
                denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
                return numerator / denominator # compute and return the fraction

        ACp = ACt*G(r1*cAMP, r2*Dt, Km1/ACt, Km2/ACt)
        dPDEp  = r3 * cAMP * ((PDEt - PDEp) / Km3) - r4 * Et * PDEp / (Km4 + PDEp)
        dcAMP = (k1*ACp) - (k3 + k2 * PDEp) * cAMP

        return [dcAMP, dPDEp]

    # oscillating parameters
    # interesting bifurcation values at k1 = 4.5 -> stable spiral at 0.8 stable node

    # 2D model
    init_conditions = {'dcAMP': 0.32, 'dPDEp': 0.11}
    x0 = list(init_conditions.values())
    time_interval = [0, 500]

    solver = solve_ivp(cAMP_PDEp_ACp, time_interval , x0, t_eval=np.linspace(0, 500, 10000), method='RK45', atol=1e-8, rtol=1e-8)

    cAMP, PDEp = solver.y


    # Create a figure with multiple axes
    fig = plt.figure(figsize=(16, 12))
    fig.suptitle("2D Model - cAMP-PDEp Oscillations", fontsize=20)


    gs = GridSpec(4, 2, figure=fig)
    legend_position = {'bbox_to_anchor': (0.9, 1), 'loc': 'upper left'}

    # First row
    ax0 = fig.add_subplot(gs[0, 0])
    ax0.grid(True, linestyle=':', color='lightgray')
    ax0.plot(solver.t, PDEp, label='cAMP', color='#1f77b4')
    ax0.set_xlabel('Time')
    ax0.set_ylabel('Concentration')
    ax0.set_xlim(0, 50)
    ax0.legend(**legend_position, fontsize=16)

    ax1 = fig.add_subplot(gs[0, 1])
    ax1.grid(True, linestyle=':', color='lightgray')
    ax1.plot(solver.t, cAMP, label='PDEp', color='orange')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Concentration')
    ax1.set_xlim(0, 50)
    ax1.legend(**legend_position, fontsize=16)

    # Second row
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.grid(True, linestyle=':', color='lightgray')
    ax2.plot(solver.t, PDEp, label='cAMP', color='#1f77b4',linestyle='-', marker='o', markersize=3, markevery=50)
    ax2.plot(solver.t, cAMP, label='PDEp', color='orange', linestyle='--', marker='s', markersize=3, markevery=50)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Concentration')
    ax2.set_xlim(0, 50)
    ax2.legend(**legend_position, fontsize=16)

    ax3 = fig.add_subplot(gs[1, 1])
    ax3.grid(True, linestyle=':', color='lightgray')
    ax3.plot(PDEp, cAMP, label='cAMP vs PDEp', color='black')
    ax3.set_xlabel('PDEp')
    ax3.set_ylabel('cAMP')
    ax3.legend(**legend_position, fontsize=16)


    # Adjust the layout and show the plot
    fig.subplots_adjust(wspace=0.3, hspace=0.4, right=0.85)
    fig.savefig('cAMP_PDEplonger.png', dpi=300, bbox_inches='tight')
    plt.show()


w = interactive(show_sol, Km1=(0.01, 10, 0.01), Km2=(0.01, 10, 0.01), ACt=(0.01, 10, 0.01), Dt=(0.01, 10, 0.01), r1=(0.01, 10, 0.01), r2=(0.01, 10, 0.01), r3=(0.01, 10, 0.01), k1=(0.01, 10, 0.01), k2=(0.01, 10, 0.01), Km3=(0.01, 10, 0.01), PDEt=(0.01, 10, 0.01), r4=(0.01, 10, 0.01), E=(0.01, 10, 0.01), continuous_update=True)
w

interactive(children=(FloatSlider(value=9.18, description='k1', max=10.0, min=0.01, step=0.01), FloatSlider(va…